In [16]:
import pandas as pd
import numpy as np
import pickle as pk
import re
import warnings

from gensim.models import FastText
from sentence_transformers import SentenceTransformer
from tqdm.notebook import tqdm
from nltk import sent_tokenize
from sklearn.metrics.pairwise import cosine_similarity

In [11]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
import os
PATH = '/Users/yoonjeong/Desktop/Lab_project/'
os.chdir(PATH)

In [2]:
df = pd.read_csv("2.Preprocessing/fraud_prep.csv", encoding="cp949")

In [5]:
# 9만자 미만의 글만 남김
temp = df[df['판례내용_전처리'].str.len() < 90000]

In [6]:
model = SentenceTransformer('jhgan/ko-sbert-sts')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.44k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/538 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/248k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/495k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [15]:
with open("sbert_embedding.pickle", "wb") as fw:
    pk.dump(embeddings, fw)

In [30]:
user_input = "갑은 을로부터 토지를 매수하기로 계약을 체결한 후 계약금 및 중도금을 지급하고 나머지 잔금을 지급하지 않은 상태에서 을에게 형질변경 및 건축허가를 받는 데에 반드시 필요하니 부동산매도용인감증명서 및 확인서면을 갑에게 건네주면 위 용도로만 사용하겠다고 거짓말하여, 이에 속은 을로부터 즉석에서 부동산매도용인감증명서 및 등기의무자본인확인서면을 교부받은 후 이를 이용하여 갑명의로 위 토지의 소유권이전등기를 경료하였습니다. 이 경우 갑을 사기죄로 처벌할 수 있는지요?"

In [31]:
clean_user_input = re.sub("[^가-힣. ]", '', user_input)
clean_user_input

'갑은 을로부터 토지를 매수하기로 계약을 체결한 후 계약금 및 중도금을 지급하고 나머지 잔금을 지급하지 않은 상태에서 을에게 형질변경 및 건축허가를 받는 데에 반드시 필요하니 부동산매도용인감증명서 및 확인서면을 갑에게 건네주면 위 용도로만 사용하겠다고 거짓말하여 이에 속은 을로부터 즉석에서 부동산매도용인감증명서 및 등기의무자본인확인서면을 교부받은 후 이를 이용하여 갑명의로 위 토지의 소유권이전등기를 경료하였습니다. 이 경우 갑을 사기죄로 처벌할 수 있는지요'

In [32]:
user_sentence_tokened = sent_tokenize(clean_user_input)
user_sentence_tokened

['갑은 을로부터 토지를 매수하기로 계약을 체결한 후 계약금 및 중도금을 지급하고 나머지 잔금을 지급하지 않은 상태에서 을에게 형질변경 및 건축허가를 받는 데에 반드시 필요하니 부동산매도용인감증명서 및 확인서면을 갑에게 건네주면 위 용도로만 사용하겠다고 거짓말하여 이에 속은 을로부터 즉석에서 부동산매도용인감증명서 및 등기의무자본인확인서면을 교부받은 후 이를 이용하여 갑명의로 위 토지의 소유권이전등기를 경료하였습니다.',
 '이 경우 갑을 사기죄로 처벌할 수 있는지요']

In [33]:
with open("sbert_embedding.pickle", 'rb') as fr:
    sbert_embedding = pk.load(fr)

In [34]:
embedding = None
count = 0
for sentence in user_sentence_tokened:
    if embedding is None:
        embedding = model.encode(sentence)
    else:
        embedding += model.encode(sentence)
    count += 1

s_embedding = embedding / count

In [35]:
s_similarity = cosine_similarity([s_embedding],sbert_embedding )
s_similarity = list(enumerate(s_similarity[0]))
s_similarity = sorted(s_similarity, key=lambda x : x[1], reverse = True)

In [36]:
precidents = []

for i in range(5):
    idx = s_similarity[i][0]
    sim = round(s_similarity[i][1] * 100, 2)

    precidents.append([df['판례일련번호'][idx], df['사건명'][idx], df['사건종류명'][idx], df['판례내용'][idx], sim])

In [37]:
s_result = pd.DataFrame(precidents, columns=['판례일련번호', '사건명', '사건종류명', '판례내용', '유사도'])
s_result

,판례일련번호,사건명,사건종류명,판례내용,유사도
0,68844,관세법위반·사기,형사,【피 고 인】 【상 고 인】 피고인【변 호 인】 변호사 한창석외 1인【원심판결】 인...,65.71
1,182109,폭력행위등처벌에관한법률위반(상습공갈)(인정된죄명상습공갈)·주민등록법위반·전자금융거래...,형사,"【피 고 인】 【항 소 인】 피고인【검 사】 문지선, 장성훈, 유시동, 나의엽...",65.21
2,101864,사기,형사,【피 고 인】 【상 고 인】 검사【원심판결】 \n전주지방법원 1985.10.12...,65.20
3,104923,"사기,횡령",형사,【피 고 인】 【상 고 인】 피고인 및 검사【변 호 인】 변호사 이상수 외 ...,64.79
4,108753,"사기,특정경제범죄가중처벌등에관한법률위반",형사,【피 고 인】 【상 고 인】 피고인【변 호 인】 변호사 한경수 외 1인【원심...,64.54


In [38]:
s_result.판례내용[0]

'【피 고 인】 【상 고 인】 피고인【변 호 인】 변호사 한창석외 1인【원심판결】 인천지법 2007. 4. 19. 선고 2006노2548, 2007노558 판결【주    문】상고를 기각한다.  상고 후의 구금일수 중 70일을 본형에 산입한다.【이    유】상고이유를 본다.  1. \n        관세법 제234조 제3호에 의하여 화폐·채권 기타 유가증권의 위조품·변조품 또는 모조품의 수출입이 금지되는바, 이때 유가증권이란 증권상에 표시된 재산상의 권리의 행사와 처분에 그 증권의 점유를 필요로 하는 것을 총칭하는 것으로서 재산권이 증권에 화체된다는 것과 그 권리의 행사와 처분에 증권의 점유를 필요로 한다는 두 가지 요소를 갖추면 족하지 반드시 유통성을 가질 필요는 없고, 또한 위 유가증권은 일반인이 진정한 것으로 오신할 정도의 형식과 외관을 갖추고 있으면 되는 것이다\n        (\n        대법원 2001. 8. 24. 선고 2001도2832 판결 참조).\n        기록에 의하면, 이 사건 잔고확인증은 일본국 대장대신이 발행한 것으로서 금액 및 수취인 기재 아래에 “국채환부금의 상환잔액이 상기와 같이 남아있음을 확인한다. 상기의 상환잔존금에 대해서 이부국고채권(15년)을 같은 금액으로 인도한다. 이 국채는 일본은행의 본ㆍ지점 등에서 취급한다.”는 내용의 기재와 함께 ‘제57회’, ‘A제1487호’ 등의 일련번호가 있는 사실, 이 사건 잔고확인증을 일본에서 교부받아 피고인이 부회장으로 근무하는 ‘\n        (이름 생략) 재단’에 기증하였다는 \n        공소외 1 및 피고인이 수사기관에서 이 사건 잔고확인증을 일본국채라고 일치하여 진술한 사실, \n        공소외 1이 이 사건 잔고확인증의 진정성을 소명하기 위하여 소지하고 있다는 ‘제57회 환부금잔고확인증교부경과개요’라는 서류에 “동 확인증이 법률적으로 국가채무증권이 된다.”는 기재가 있는 사실 등을 알 수 있고, 이에 따르면 이 사건 잔고확인증은 이부국고채권으로 교환할 수